In [55]:
!wget -O "guthenberg-cs.txt" "https://www.gutenberg.org/cache/epub/34225/pg34225.txt"
!wget -O "guthenberg-en.txt" "https://www.gutenberg.org/cache/epub/37536/pg37536.txt"

--2026-01-04 20:10:25--  https://www.gutenberg.org/cache/epub/34225/pg34225.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 769766 (752K) [text/plain]
Saving to: ‘guthenberg-cs.txt’

guthenberg-cs.txt   100%[===================>] 751,72K  1,27MB/s    in 0,6s    

2026-01-04 20:10:26 (1,27 MB/s) - ‘guthenberg-cs.txt’ saved [769766/769766]

--2026-01-04 20:10:26--  https://www.gutenberg.org/cache/epub/37536/pg37536.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 724943 (708K) [text/plain]
Saving to: ‘guthenberg-en.txt’

guthenberg-en.txt   100%[===================>] 707,95K  1,02MB/s    in 0,7s    

2026-01

In [56]:
%pip install sacremoses

from sacremoses import MosesTokenizer, MosesDetokenizer
import os
from collections import Counter, defaultdict
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [57]:
filenames = {"cs": "guthenberg-cs.txt","en": "guthenberg-en.txt"}


In [58]:
for name in filenames:
    print(f"{name}: {os.path.getsize(filenames[name])} bytes")

cs: 769766 bytes
en: 724943 bytes


In [59]:
corpus_full = {}

for lang in filenames:
    with open(filenames[lang], "r") as f:
        corpus_full[lang] = f.read()

data = {lang: MosesTokenizer(lang).tokenize(corpus_full[lang]) for lang in corpus_full}

In [60]:
start_special_symbol, end_special_symbol = "<s>", "</s>"

get_unigrams = lambda tokens : Counter(tokens + [start_special_symbol, end_special_symbol])
get_bigrams = lambda tokens : Counter(zip([start_special_symbol] + tokens, tokens + [end_special_symbol]))
get_trigrams = lambda tokens : Counter(zip(2 * [start_special_symbol] + tokens, [start_special_symbol] + tokens + [end_special_symbol], tokens + 2 * [end_special_symbol]))

In [61]:
print("Length of english tokenized data:", len(data["en"]))
data["en"] = data["en"][:30_000]
print("Length of czech tokenized data:", len(data["cs"]))
data["cs"] = data["cs"][:15_000]
print("Unique tokens in english 30k dataset:", len(set(data["en"])))
print("Unique tokens in czech 15k dataset:", len(set(data["cs"])))

Length of english tokenized data: 149191
Length of czech tokenized data: 136966
Unique tokens in english 30k dataset: 4426
Unique tokens in czech 15k dataset: 4950


In [62]:
uni = {lang: get_unigrams(data[lang]) for lang in data}
bi = {lang: get_bigrams(data[lang]) for lang in data}

print("English tokens with at least 50 occurrences:", len([token for token, count in uni["en"].items() if count >= 50]))
print("Czech tokens with at least 20 occurrences:", len([token for token, count in uni["cs"].items() if count >= 20]))

English tokens with at least 50 occurrences: 73
Czech tokens with at least 20 occurrences: 77


In [72]:
def get_class_bigrams(word2class, bigrams):
    class_bigrams = defaultdict(int)
    for (w1, w2), count in bigrams.items():
        c1, c2 = word2class[w1], word2class[w2]
        class_bigrams[(c1, c2)] += count
    return class_bigrams

def get_class_bigrams_matrix(word2class, bigrams):
    class_bigrams_matrix = np.zeros((len(word2class), len(word2class)), dtype=int)
    for (l, r), count in bigrams.items():
        if l not in word2class or r not in word2class:
            continue
        c1, c2 = word2class[l], word2class[r]
        class_bigrams_matrix[c1, c2] += count
    return class_bigrams_matrix

def q(classes_matrix,l,r,N): # q_k(l,r)
    c_k = classes_matrix[l,r]
    c_kl = classes_matrix[l,:].sum()
    c_kr = classes_matrix[:,r].sum()
    if c_kl == 0 or c_kr == 0 or c_k == 0:
        return 0.0
    return c_k / N * np.log(N * c_k / (c_kl * c_kr))

# ok
def mutual_information(classes_matrix, N):
    mi = 0
    for i in range(classes_matrix.shape[0]):
        for j in range(classes_matrix.shape[1]):
            mi += q(classes_matrix, i, j, N)
    return mi

def test_merge(c, a, b, N):
    """
    c : matrix[i,j] = # bigrams class i -> class j
    a, b : classes to merge
    should return mutal information of something
    """

    # calculate q on row a,b and column a,b
    # create new matrix with merged row/column
    # calculate q on merged row/column
    # return the difference or whatever

    value = 0.0
    for i in range(c.shape[0]):
        value -= q(c, i, a, N)
        value -= q(c, i, b, N)
        value -= q(c, a, i, N)
        value -= q(c, b, i, N)

    value -= q(c, a, a, N)
    value -= q(c, b, b, N)
    value -= q(c, a, b, N)
    value -= q(c, b, a, N)
    new_matrix = np.copy(c)
    if a < b:
        new_matrix[:,a] += new_matrix[:,b]
        new_matrix[a,:] += new_matrix[b,:]
        new_matrix = np.delete(new_matrix, b, axis=0)
        new_matrix = np.delete(new_matrix, b, axis=1)

        for i in range(new_matrix.shape[0]):
            value += q(new_matrix, a, i, N)
            value += q(new_matrix, i, a, N)

        value -= q(new_matrix, a, a, N)
    else:
        new_matrix[:,b] += new_matrix[:,a]
        new_matrix[b,:] += new_matrix[a,:]
        new_matrix = np.delete(new_matrix, a, axis=0)
        new_matrix = np.delete(new_matrix, a, axis=1)

        for i in range(new_matrix.shape[0]):
            value += q(new_matrix, b, i, N)
            value += q(new_matrix, i, b, N)
        value -= q(new_matrix, b, b, N)

    return new_matrix, value



def word_classes(initial_words, unigrams, bigrams, target_number=15):
    N = sum(bigrams.values())                                               # TODO change : total number of tokens
    word2class = {w:i for i, (w,count) in enumerate(initial_words.items())} # r function word2class[word] = class_id
    initial_w2c= word2class.copy()
    c = get_class_bigrams_matrix(word2class, bigrams)                       # i,j position -> #bigrams s.t. class i-> class j

    # N = np.sum(class_bigrams_matrix)
    
    mi = mutual_information(c, N)
    print(f"Initial mutual information is : {mi}")
    history = []

    while c.shape[0] > target_number:

        best_diff, best_pair, best_matrix = -float("inf"), None, None
        # new_mi = mi + mi_diff

        K = c.shape[0] # current number of classes
        mi = mutual_information(c, N)

        for a in range(K):
            for b in range(a+1, K):
                if a == b:
                    continue
                merged_matrix, diff = test_merge(c, a, b, N)

                if diff > best_diff:
                    best_diff = diff
                    best_pair = (a, b)
                    best_matrix = merged_matrix

        # apply best merge
        # just apply merge to word2class
        a, b = best_pair
        classA = [word for word, c in word2class.items() if c == a]
        classB = [word for word, c in word2class.items() if c == b]

        for w,c in word2class.items():
            if c == b:
                word2class[w] = a
            elif c > b:
                word2class[w] = c - 1

        c = best_matrix
        mi += best_diff

        print(
            f"Number of classes {c.shape[0]}, mi={mi:.6f}"
        )
        print(f"Merged classes :\n{classA}\n{classB}")


    return c, word2class



m, w2c = word_classes({w:c for w,c in get_unigrams(data["en"]).items() if c >= 50}, get_unigrams(data["en"]), get_bigrams(data["en"]), target_number=15)


Initial mutual information is : 0.7192609886464977
Number of classes 72, mi=0.718735
Merged classes :
['a']
['an']
Number of classes 71, mi=0.718206
Merged classes :
['this']
['my']
Number of classes 70, mi=0.717622
Merged classes :
['convicts']
['prisoners']
Number of classes 69, mi=0.716940
Merged classes :
['He']
['It']
Number of classes 68, mi=0.716157
Merged classes :
['are']
['were']
Number of classes 67, mi=0.715365
Merged classes :
['little']
['day']
Number of classes 66, mi=0.714573
Merged classes :
['this', 'my']
['their']
Number of classes 65, mi=0.713756
Merged classes :
['would']
['could']
Number of classes 64, mi=0.712848
Merged classes :
['himself']
['been']
Number of classes 63, mi=0.711908
Merged classes :
['at']
['from']
Number of classes 62, mi=0.710913
Merged classes :
['will']
['would', 'could']
Number of classes 61, mi=0.709835
Merged classes :
['very']
['up']
Number of classes 60, mi=0.708760
Merged classes :
['him']
['me']
Number of classes 59, mi=0.707694
Merge

In [64]:
asd = "asdfghjklk"
a = {a:i for i,a in enumerate(asd)}
a

{'a': 0, 's': 1, 'd': 2, 'f': 3, 'g': 4, 'h': 5, 'j': 6, 'k': 9, 'l': 8}

In [65]:
for w,c in a.items():
    a[w] = c * 2
a

{'a': 0, 's': 2, 'd': 4, 'f': 6, 'g': 8, 'h': 10, 'j': 12, 'k': 18, 'l': 16}

In [66]:
# [a for a,b in w2c.items() if b == 181]


In [67]:
# np.where(m == 144)

In [68]:
# !wget "https://ufallab.ms.mff.cuni.cz/~helcl/npfl147/cc.en.50.bin"
# !wget "https://ufallab.ms.mff.cuni.cz/~helcl/npfl147/cc.cs.50.bin"
# import fasttext
# ft_en = fasttext.load_model('cc.en.50.bin')